# CSCA 5632 Assignment #4 - NLP Kaggle BBC News Classification Mini Projecy
### By Moshiur Howlader
##### Github Link : https://github.com/Mosh333/csca5632-nlp-kaggle-bbc-news-classification

In [ ]:
import os

# Always start paths relative to the notebook file location
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(BASE_DIR, "data")

print("Base directory:", BASE_DIR)
print("Data directory:", DATA_DIR)

NameError: name '__file__' is not defined